<a href="https://colab.research.google.com/github/neekonsu/Thyroid-Clustering/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cmapPy
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from sklearn import preprocessing
import gzip
import shutil
import gc
# for csv dataframe use pandas
import pandas as pd
# from eqgen_exp
import matplotlib.pyplot as plt
# for floor function
import math
# for image processing, we import openCV
import cv2
# for interfacing with filesystem
import os
# for using randomness
import random
# for listing progress bars where we need them
from tqdm import tqdm
# for converting png to jpg
from PIL import Image
# for building recursive file array
from glob import glob 
import numpy as np
import cmapPy as gt
# FORMAT MATPLOTLIB
%matplotlib inline
gc.collect()

     |████████████████████████████████| 153kB 4.8MB/s 


Using TensorFlow backend.


8

In [2]:
try:
  assert os.listdir('/content/Thyroid-Clustering')
except:
  !git clone "https://github.com/neekonsu/Thyroid-Clustering"
  !wget "https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz"
gc.collect()

Cloning into 'Thyroid-Clustering'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
--2019-06-11 17:33:01--  https://storage.googleapis.com/gtex_analysis_v7/rna_seq_data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 867038648 (827M) [application/gzip]
Saving to: ‘GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz’

GTEx_Analysis_2016- 100%[===================>] 826.87M   120MB/s    in 7.3s    

2019-06-11 17:33:08 (113 MB/s) - ‘GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz’ saved [867038648/867038648]



60

In [3]:
with gzip.open('/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz', 'rb') as f_in:
    with open('/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
gc.collect()
!rm "/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz"
!rm -r "/content/Thyroid-Clustering"
gc.collect()

0

In [0]:
# with open("/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct", 'r') as fin:
#     d = fin.read().splitlines(True)
# with open("/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct", 'w') as fout:
#     fout.writelines(d[2:])
!tail -n +2 "/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct" > "/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.tmp" && mv "/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.tmp" "/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct"

In [0]:
gc.collect()
fields=[]
data = pd.read_csv('/content/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct', sep='\t', usecols=fields)
# Preview the first 5 lines of the loaded data 
data.head()

In [0]:
data.tail()

In [0]:
data.shape

In [0]:
data = data.drop(labels=[
    "SAMPID",
    "SMPTHNTS",
    "SMCENTER",
    "SMTS",
    "SMTSD",
    "SMNABTCH",
    "SMNABTCHT",
    "SMNABTCHD",
    "SMGEBTCH",
    "SMGEBTCHD",
    "SMGEBTCHT",
    "SMAFRZE",
    "SMGTC",
    "SMCHMPRS",
    "SMNUMGPS",
    "SM550NRM",
    "SMUNMPRT",
    "SM350NRM",
    "SMRDLGTH",
    "SMMNCPB",
    "SMESTLBS",
    "SMMNCV",
    "SMCGLGTH",
    "SMGAPPCT",
    "SMUNPDRD",
    "SMNUM5CD",
    "SMDPMPRT"], 
          axis=1)

In [0]:
data = pd.concat([data,pd.get_dummies(data["SMATSSCR"],prefix=["SMATSSCR"], drop_first=True)],axis=1)
data = pd.concat([data,pd.get_dummies(data["SMUBRID"],prefix=["SMUBRID"], drop_first=True)],axis=1)
data.drop(["SMATSSCR"],axis=1, inplace=True)
data.drop(["SMUBRID"],axis=1, inplace=True)
data.head()

In [0]:
# Get column names first
names = data.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(data)
data = pd.DataFrame(scaled_df, columns=names)

data.head()